In [ ]:
%load_ext autoreload
%autoreload 2
%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/

import os
import pickle
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

import time
import optuna
import argparse


from gluonts.evaluation import make_evaluation_predictions
from gluonts.mx.trainer import Trainer
from gluonts.mx.trainer.callback import TrainingHistory
from gluonts.mx.distribution import StudentTOutput, MultivariateGaussianOutput
from sklearn.metrics import mean_absolute_error
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

from gluonts.mx.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.mx.model.transformer import TransformerEstimator
from gluonts.mx.model.deepar import DeepAREstimator 
from gluonts.mx.model.wavenet import WaveNetEstimator
from gluonts.mx.model.seq2seq import MQCNNEstimator

import matplotlib.pyplot as plt
import json
from pathlib import Path
from gluonts.dataset.split import split
from gluonts.dataset.common import ListDataset
import copy

from gluonts.evaluation import Evaluator
from gluonts.evaluation import make_evaluation_predictions

from gluonts_utils import series_to_gluonts_dataset, load_params
from fracdiff import invert_fd

class Objective:
    def __init__( self, model, dataset_name, X, d=None):
        '''
        model: str
        dataset_name: str
        X: pd.DataFrame. Expects this df to have a target column called 'target', 
        and if using a frac diffed target to be inverted, also expects the original version to be named 'values_o'
        d: float. Differencing amount used if the target in X was frac diffed, None if none used.
        '''
        self.model = model
        self.dataset_name = dataset_name
        self.data_params = load_params('gluonts_params.txt', dataset_name)
        print(self.data_params)

        self.ctx = 'gpu(0)'
        self.original_value_col = 'values_o' # gets used in the inversion of fd if needed, 

        self.prediction_length = self.data_params['prediction_length']
        self.context_length = self.data_params['context_length']
        self.freq = self.data_params['freq']
        
        self.d = d
        self.X = X
        X_train_val, X_test = X[:-self.prediction_length], X[-self.prediction_length:]
        X_train, X_val = X_train_val[:-self.prediction_length], X_train_val[-self.prediction_length:]
        
        self.tuning_dataset = series_to_gluonts_dataset(X_train, X_val,  self.data_params)
        self.eval_dataset =  series_to_gluonts_dataset(X_train_val, X_test,  self.data_params)
        

    def get_params(self, trial) -> dict:
        if self.model == 'feedforward':
            return {
              "num_hidden_dimensions": [trial.suggest_int("hidden_dim_{}".format(i), 10, 100) for i in range(trial.suggest_int("num_layers", 1, 5))],
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'wavenet':
            return {
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'mqcnn':
            return {
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'deepar':
            return {
                "num_cells": trial.suggest_int("num_cells", 10, 100),
                "num_layers": trial.suggest_int("num_layers", 1, 10),
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100)
            }
        elif self.model == 'transformer':
            # num_heads must divide model_dim
            valid_pairs = [ (i,d) for i in range(10,101) for d in range(1,11) if i%d == 0  ]
            model_dim_num_heads_pair = trial.suggest_categorical("model_dim_num_heads_pair", valid_pairs)

            return {
                "inner_ff_dim_scale": trial.suggest_int("inner_ff_dim_scale", 1, 5),
                "model_dim": model_dim_num_heads_pair[0],
                "embedding_dimension": trial.suggest_int("embedding_dimension", 1, 10),
                "num_heads": model_dim_num_heads_pair[1],
                "dropout_rate": trial.suggest_uniform("dropout_rate", 0.0, 0.5),
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        
    def load_model(self, params):
        history = TrainingHistory()
        if self.model == 'feedforward':
            estimator = SimpleFeedForwardEstimator(
                num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
                prediction_length=self.prediction_length,
                context_length=self.context_length,
                batch_normalization=False,
                mean_scaling=False,
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )
        elif self.model == 'wavenet':
            estimator = WaveNetEstimator(
                freq=self.freq,
                prediction_length=self.prediction_length,
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                    num_batches_per_epoch=100, callbacks=[history], add_default_callbacks=False),
            )
        elif self.model == 'mqcnn':
            estimator = MQCNNEstimator(
                freq=self.freq,
                prediction_length=self.prediction_length,
                context_length=self.context_length,
                distr_output=StudentTOutput(),
                quantiles=None,
                scaling=False, 
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history], hybridize=False),
            )
        elif self.model == 'deepar':
            estimator = DeepAREstimator(
                freq=self.freq,
                context_length=self.context_length,
                distr_output=StudentTOutput(),
                prediction_length=self.prediction_length,
                # num_cells= params['num_cells'],
                # num_layers= params['num_layers'],
                scaling=False, # True by default
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )
        elif self.model == 'transformer':
            estimator = TransformerEstimator(
                freq=self.freq,
                context_length=self.context_length,
                prediction_length=self.prediction_length,
                distr_output=StudentTOutput(),
                inner_ff_dim_scale= params['inner_ff_dim_scale'],
                model_dim= params['model_dim'],
                embedding_dimension= params['embedding_dimension'],
                num_heads= params['num_heads'],
                dropout_rate= params['dropout_rate'],
                # scaling=False, # True by default False
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )

        return estimator, history

    def train_test(self, params, tuning=True):
        model, history = self.load_model(params)

        if tuning:
            predictor = model.train(self.tuning_dataset.train, self.tuning_dataset.test)
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=self.tuning_dataset.test,
                predictor=predictor,
            )
        else:
            predictor = model.train(self.eval_dataset.train)
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=self.eval_dataset.test,
                predictor=predictor,
            )

        forecasts = list(forecast_it)
        tss = list(ts_it)
        # invert here if fd
        if self.d:
            new_fd_series = self.eval_dataset.train[0]['target']
            original_series = self.X[self.original_value_col][:-self.prediction_length]

            unfd_samples = []
            for predsampleset in forecasts[0].samples:
                unfd_sampleset = []
                fd = new_fd_series.copy()
                o = original_series.copy()
                for pred in predsampleset:
                    fd = np.append(fd, pred)
                    ufd = invert_fd(pd.Series(fd), o, self.d)
                    unfd_pred = ufd.iloc[-1]
                    unfd_sampleset.append(unfd_pred)
                    o = pd.Series(np.append(o, unfd_pred))
                unfd_samples.append([unfd_sampleset])
            forecasts[0].samples = np.array(unfd_samples).squeeze()
            tss[0][0] = self.X[self.original_value_col].values
        evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
        agg_metrics, item_metrics = evaluator(tss, forecasts)
        
        print(f'#####__tuning mase = {agg_metrics["RMSE"]}__###### ')
        return agg_metrics, predictor, history
        

    def __call__(self, trial):

        params = self.get_params(trial)

        agg_metrics, _, _ = self.train_test(params, tuning=True)

        return agg_metrics['RMSE']

/mnt/c/Users/resha/Documents/Github/balancing_framework


In [2]:
X = pd.read_csv('m4_1165_fd.csv') # original fd, default thresh, no skips
X_withoutfd = X.drop(columns=['values_fd']).rename(columns={"values_o":"target"})
X_fdtarget = X.drop(columns=['values_o']).rename(columns={"values_fd": "target"})
X = X.rename(columns={"values_fd": "target"})

In [3]:

def run(X, model, dataset_name, save_label, n_trials, n_repeats):
    start_time = time.perf_counter()

    # run tuning
    study = optuna.create_study(direction="minimize")
    obj = Objective(
        model=model,
        dataset_name=dataset_name,
        X=X
        )
    study.optimize(
        obj,
        n_trials=n_trials,
    )
    trial = study.best_trial

    # print results
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    print(f'Runtime: {time.perf_counter() - start_time}')

    # unpack params for next runs
    if model == 'feedforward':
        trial.params["num_hidden_dimensions"] = [ trial.params[f"hidden_dim_{i}"] for i in range(trial.params["num_layers"]) ]
    elif model == 'transformer':
        trial.params["model_dim"] = trial.params["model_dim_num_heads_pair"][0]
        trial.params["num_heads"] = trial.params["model_dim_num_heads_pair"][1]

    # repeat best run 5 times
    mases = []
    smapes = []
    rmses = []
    params_sets = []
    save_dir = f'results/monash_gluonts_single_tuned_runs/{model}_{dataset_name}_{save_label}_{n_trials}_trials'
    os.makedirs(save_dir, exist_ok=True)
    for i in range(n_repeats):
        res, predictor, history = obj.train_test(trial.params, tuning=False)

        # plot and save training history
        plt.plot(history.loss_history, label='Training Loss')
        plt.plot(history.validation_loss_history, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Learning Curve')
        plt.legend()
        plt.grid(True)

        # Save the figure
        plt.savefig(f'{save_dir}/learning_curve_{i}.png')
        # save the history values
        with open(f'{save_dir}/loss_history_{i}.json', "w") as f:
            json.dump(history.loss_history, f)
        # Clear the current figure
        plt.clf()
        mases.append(res['MASE'])
        smapes.append(res['sMAPE'])
        rmses.append(res['RMSE'])
        params_sets.append(trial.params)


    mase_mean = np.array(mases).mean()
    mase_std = np.std(np.array(mases))
    rmses_mean = np.array(rmses).mean()
    rmses_std = np.std(np.array(rmses))
    smape_mean = np.array(smapes).mean()
    smape_std = np.std(np.array(smapes))

    print(f'##### MASE MEAN: {mase_mean} MASE STD: {mase_std}')
    print(f'##### sMAPE MEAN: {smape_mean} sMAPE STD: {smape_std}')
    print(f'##### RMSE MEAN: {rmses_mean} RMSE STD: {rmses_std}')


    # save best params to json
    with open(f'{save_dir}/params.json', "w") as f:
        json.dump(trial.params, f)

    # save the last predictor
    os.makedirs(f'{save_dir}/predictor', exist_ok=True)
    predictor.serialize(Path(f"{save_dir}/predictor"))

    end_time = time.perf_counter()
    runtime = (end_time - start_time) / 60

    file_path = "gluonts_results.txt"
    with open(file_path, "a") as file:
        file.write(
            f'''\n\n\n\n {model} {n_trials} tuning trials {n_repeats} repeat test evals on {dataset_name} {save_label} data form, Runtime: {runtime} minutes 
            \n Params: {trial.params}
            \n MASE MEAN: {mase_mean} MASE STD: {mase_std}
            \n sMAPE MEAN: {smape_mean} sMAPE STD: {smape_std}
            \n RMSE MEAN: {rmses_mean} RMSE STD: {rmses_std}
            '''
        )

In [4]:

model = 'transformer'
dataset_name = 'm4_daily_dataset'
params = {'model_dim_num_heads_pair': (60, 1), 'inner_ff_dim_scale': 1, 'embedding_dimension': 1, 'dropout_rate': 0.49740211005822393, 'trainer:learning_rate': 0.00016085457714081904, 'trainer:epochs': 1, 'model_dim': 60, 'num_heads': 1}

study = optuna.create_study(direction="minimize")
obj = Objective(
    model=model,
    dataset_name=dataset_name,
    X=X,
    d=0.1
    )

res, predictor, history = obj.train_test(params, tuning=False)

[I 2025-10-08 11:23:05,136] A new study created in memory with name: no-name-6d456539-783e-47e4-9511-0b488a4a6186


[11:23:05] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8907 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


{'freq': 'D', 'prediction_length': 14, 'context_length': 90}


100%|██████████| 100/100 [00:08<00:00, 12.35it/s, epoch=1/1, avg_epoch_loss=9.63]


[7324.4    6469.2603 6146.632  ... 3508.9778 3435.4717 3410.9807] 0       7324.4
1       7201.7
2       7196.4
3       7177.0
4       7140.6
         ...  
8514    9026.8
8515    9020.2
8516    9006.4
8517    9022.3
8518    8949.9
Name: values_o, Length: 8519, dtype: float64 0.1
0       7324.400000
1       7201.700000
2       7196.400000
3       7177.000000
4       7140.600000
           ...     
8515    9020.200000
8516    9006.400000
8517    9022.300000
8518    8949.900000
8519    8918.754104
Name: values_o, Length: 8520, dtype: float64
[ 7324.4     6469.2603  6146.632  ...  3435.4717  3410.9807 -1397.052 ] 0       7324.400000
1       7201.700000
2       7196.400000
3       7177.000000
4       7140.600000
           ...     
8515    9020.200000
8516    9006.400000
8517    9022.300000
8518    8949.900000
8519    8918.754104
Length: 8520, dtype: float64 0.1
0       7324.400000
1       7201.700000
2       7196.400000
3       7177.000000
4       7140.600000
           ...     
8516    90

Running evaluation: 1it [00:00,  1.56it/s]

#####__tuning mase = 1437.7706219244867__###### 


In [5]:
forecast_it, ts_it = make_evaluation_predictions(
                dataset=obj.eval_dataset.test,
                predictor=predictor,
            )

forecasts = list(forecast_it)
tss = list(ts_it)
# invert here if fd
# evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
# agg_metrics, item_metrics = evaluator(tss, forecasts)

In [6]:
new_fd_series = obj.eval_dataset.train[0]['target']
original_series = obj.X['values_o'][:-obj.prediction_length]

unfd_samples = []
for predsampleset in forecasts[0].samples:
    unfd_sampleset = []
    fd = new_fd_series.copy()
    o = original_series.copy()
    for pred in predsampleset:
        fd = np.append(fd, pred)
        ufd = invert_fd(pd.Series(fd), o, obj.d)
        unfd_pred = ufd.iloc[-1]
        unfd_sampleset.append(unfd_pred)
        o = pd.Series(np.append(o, unfd_pred))
    unfd_samples.append([unfd_sampleset])
forecasts[0].samples = np.array(unfd_samples).squeeze()
tss[0][0] = obj.X['values_o'].values


In [12]:
tss

[                 0
 2025-10-08  7324.4
 2025-10-09  7201.7
 2025-10-10  7196.4
 2025-10-11  7177.0
 2025-10-12  7140.6
 ...            ...
 2049-02-12  8862.1
 2049-02-13  8849.8
 2049-02-14  8870.6
 2049-02-15  8881.3
 2049-02-16  8874.1
 
 [8533 rows x 1 columns]]

In [11]:
forecasts[0].samples.shape

(100, 14)

In [7]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
evaluator(tss, forecasts)

Running evaluation: 1it [00:01,  1.24s/it]


({'MSE': 3710102.5526367957,
  'abs_error': 28523.040101571234,
  'abs_target_sum': 124738.70000000001,
  'abs_target_mean': 8909.907142857144,
  'seasonal_error': 40.61395867574548,
  'MASE': 50.16403408298618,
  'MAPE': 0.22874236546589963,
  'sMAPE': 0.20445834872386268,
  'MSIS': 495.2101582734275,
  'num_masked_target_values': 0.0,
  'QuantileLoss[0.1]': 7916.051886927291,
  'Coverage[0.1]': 0.0,
  'QuantileLoss[0.5]': 28523.040101571234,
  'Coverage[0.5]': 1.0,
  'QuantileLoss[0.9]': 18939.547288640213,
  'Coverage[0.9]': 1.0,
  'RMSE': 1926.162649579935,
  'NRMSE': 0.21618212386467942,
  'ND': 0.22866231651902122,
  'wQuantileLoss[0.1]': 0.06346107412476874,
  'wQuantileLoss[0.5]': 0.22866231651902122,
  'wQuantileLoss[0.9]': 0.15183377162532727,
  'mean_absolute_QuantileLoss': 18459.546425712913,
  'mean_wQuantileLoss': 0.1479857207563724,
  'MAE_Coverage': 0.5,
  'OWA': nan},
   item_id forecast_start           MSE     abs_error  abs_target_sum  \
 0       0     2049-01-15  3.

In [5]:

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
evaluator(tss, forecasts)

Running evaluation: 0it [00:00, ?it/s]

Running evaluation: 1it [00:01,  1.37s/it]


({'MSE': 2389461.1428571427,
  'abs_error': 23005.43359375,
  'abs_target_sum': 47793.44140625,
  'abs_target_mean': 3413.8172433035716,
  'seasonal_error': 41.30452791148157,
  'MASE': 39.78365907529595,
  'MAPE': 0.4816217081887381,
  'sMAPE': 0.3833823544638498,
  'MSIS': 481.2921943473904,
  'num_masked_target_values': 0.0,
  'QuantileLoss[0.1]': 8742.343896484375,
  'Coverage[0.1]': 0.0,
  'QuantileLoss[0.5]': 23005.433349609375,
  'Coverage[0.5]': 1.0,
  'QuantileLoss[0.9]': 17673.177246093746,
  'Coverage[0.9]': 1.0,
  'RMSE': 1545.7881946945845,
  'NRMSE': 0.4528034409946081,
  'ND': 0.48135126738836503,
  'wQuantileLoss[0.1]': 0.1829193219666565,
  'wQuantileLoss[0.5]': 0.48135126228011965,
  'wQuantileLoss[0.9]': 0.36978247906171086,
  'mean_absolute_QuantileLoss': 16473.65149739583,
  'mean_wQuantileLoss': 0.34468435443616235,
  'MAE_Coverage': 0.5,
  'OWA': nan},
   item_id forecast_start           MSE     abs_error  abs_target_sum  \
 0       0     2049-01-15  2.389461e+06